In [ ]:
# Setup
import sys
from pathlib import Path

notebook_dir = Path.cwd()
project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from core import (
    get_policy_by_type,
    PolicyType,
    simulate_healthcare_years,
    list_available_policies,
    HealthcarePolicyFactory,
)
from core.comparison import compare_policies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("✅ Setup complete!")
print(f"\n📋 Available Policies: {list_available_policies()}")

---

## 1. Current US Healthcare System

The US healthcare system is unique among developed nations:

| Metric | US (2024) | OECD Average |
|--------|-----------|---------------|
| Spending (% GDP) | 18.5% | 10.5% |
| Per Capita | $13,000 | $5,000 |
| Uninsured | ~8% | <2% |
| Life Expectancy | 76 years | 80 years |

### Coverage Breakdown (2024)
- **Employer-Sponsored**: 55%
- **Medicare**: 19% (65+ and disabled)
- **Medicaid**: 18% (low-income)
- **Individual/ACA**: 6%
- **Uninsured**: 8%

In [ ]:
# Load and examine Current US policy baseline
current_us = get_policy_by_type(PolicyType.CURRENT_US)

print(f"📋 Policy: {current_us.policy_name}")
print(f"📝 Description: {current_us.description[:200]}...")
print(f"\n🎯 Key Parameters:")
print(f"   • Coverage: {current_us.coverage_percentage:.0%}")
print(f"   • Spending Target: {current_us.healthcare_spending_target_gdp:.1%} of GDP")
print(f"   • Admin Overhead: {current_us.admin_overhead_percentage:.1%}")

---

## 2. Compare Healthcare Policy Models

PoliSim includes 8 distinct healthcare policy models:

| Policy | Type | Key Features |
|--------|------|---------------|
| **USGHA** | Universal | US Global Health Assurance - comprehensive reform |
| **CURRENT_US** | Baseline | Current mixed system (employer + Medicare + Medicaid) |
| **M4A** | Single-Payer | Medicare-for-All (Sanders-style) |
| **UK_NHS** | National | UK National Health Service model |
| **CANADA** | Single-Payer | Canadian provincial single-payer |
| **AUSTRALIA** | Hybrid | Australian Medicare + private option |
| **GERMANY** | Multi-Payer | German statutory/private system |
| **UN_FRAMEWORK** | Universal | UN Healthcare Framework model |

In [ ]:
# Load all policies and compare key parameters
policies = {
    'USGHA': PolicyType.USGHA,
    'Current US': PolicyType.CURRENT_US,
    'Medicare-for-All': PolicyType.M4A,
    'UK NHS': PolicyType.UK_NHS,
    'Canada': PolicyType.CANADA,
    'Australia': PolicyType.AUSTRALIA,
    'Germany': PolicyType.GERMANY,
}

comparison_data = []
for name, policy_type in policies.items():
    policy = get_policy_by_type(policy_type)
    comparison_data.append({
        'Policy': name,
        'Coverage %': f"{policy.coverage_percentage:.0%}",
        'Target % GDP': f"{policy.healthcare_spending_target_gdp:.1%}",
        'Admin Overhead': f"{policy.admin_overhead_percentage:.1%}",
    })

df_comparison = pd.DataFrame(comparison_data)
print("📊 Healthcare Policy Comparison")
print("=" * 60)
print(df_comparison.to_string(index=False))

In [ ]:
# Run 10-year simulations for multiple policies
print("🔄 Running simulations for 4 policies...\n")

selected_policies = ['USGHA', 'Current US', 'Medicare-for-All', 'UK NHS']
results = {}

for name in selected_policies:
    policy = get_policy_by_type(policies[name])
    results[name] = simulate_healthcare_years(
        policy=policy,
        base_gdp=29e12,
        initial_debt=35e12,
        years=10,
        population=335e6,
        gdp_growth=0.025,
        start_year=2025
    )
    print(f"   ✅ {name}: {len(results[name])} years projected")

print("\n📊 All simulations complete!")

In [ ]:
# Visualize policy comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Healthcare Policy Comparison: 10-Year Projections', fontsize=16, fontweight='bold')

colors = {'USGHA': '#1976D2', 'Current US': '#E53935', 'Medicare-for-All': '#388E3C', 'UK NHS': '#FF9800'}

# Plot 1: Healthcare Spending ($)
ax1 = axes[0, 0]
for name, df in results.items():
    ax1.plot(df['Year'], df['Health Spending ($)'] / 1e12, linewidth=2, label=name, color=colors[name])
ax1.set_xlabel('Year')
ax1.set_ylabel('$ Trillions')
ax1.set_title('Healthcare Spending')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Healthcare as % of GDP
ax2 = axes[0, 1]
for name, df in results.items():
    ax2.plot(df['Year'], df['Health % GDP'] * 100, linewidth=2, label=name, color=colors[name])
ax2.set_xlabel('Year')
ax2.set_ylabel('% of GDP')
ax2.set_title('Healthcare as % of GDP')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Revenue Collection
ax3 = axes[1, 0]
for name, df in results.items():
    ax3.plot(df['Year'], df['Revenue ($)'] / 1e12, linewidth=2, label=name, color=colors[name])
ax3.set_xlabel('Year')
ax3.set_ylabel('$ Trillions')
ax3.set_title('Revenue Collection')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Final Year Comparison Bar Chart
ax4 = axes[1, 1]
final_spending = [results[name].iloc[-1]['Health % GDP'] * 100 for name in selected_policies]
bars = ax4.bar(selected_policies, final_spending, color=[colors[n] for n in selected_policies])
ax4.set_ylabel('% of GDP')
ax4.set_title('Healthcare Spending in 2034 (% GDP)')
ax4.bar_label(bars, fmt='%.1f%%', padding=5)

plt.tight_layout()
plt.show()

---

## 3. Understanding Policy Trade-offs

Each healthcare policy involves trade-offs:

| Policy | Cost | Coverage | Admin Efficiency | Choice |
|--------|------|----------|------------------|--------|
| USGHA | Lower | Universal | High | Moderate |
| Current US | Highest | 92% | Low | High |
| M4A | Lower | Universal | High | Limited |
| Germany | Moderate | Universal | Moderate | High |

In [ ]:
# Calculate 10-year cumulative metrics
print("📊 10-Year Cumulative Analysis")
print("=" * 70)

for name, df in results.items():
    total_spending = df['Health Spending ($)'].sum() / 1e12
    avg_pct_gdp = df['Health % GDP'].mean() * 100
    final_pct = df.iloc[-1]['Health % GDP'] * 100
    
    print(f"\n{name}:")
    print(f"   • Total 10-year spending: ${total_spending:.1f}T")
    print(f"   • Average % GDP: {avg_pct_gdp:.1f}%")
    print(f"   • Final year % GDP: {final_pct:.1f}%")

---

## 4. Try It Yourself: Custom Analysis

Compare your own selection of policies and time horizons.

In [ ]:
# 🎮 EXPERIMENT: Customize your analysis
# ========================================

# Choose two policies to compare
policy_a = get_policy_by_type(PolicyType.USGHA)     # Try: M4A, CANADA, GERMANY
policy_b = get_policy_by_type(PolicyType.CURRENT_US)

# Run simulations
years = 15  # Try: 10, 15, 22

results_a = simulate_healthcare_years(policy_a, 29e12, 35e12, years=years, start_year=2025)
results_b = simulate_healthcare_years(policy_b, 29e12, 35e12, years=years, start_year=2025)

# Compare
savings = (results_b['Health Spending ($)'].sum() - results_a['Health Spending ($)'].sum()) / 1e12

print(f"📊 {years}-Year Comparison: {policy_a.policy_name} vs {policy_b.policy_name}")
print(f"\n💰 Cumulative Savings with {policy_a.policy_name}: ${savings:.1f} Trillion")

---

## 🎓 What You've Learned

✅ The US spends 18.5% of GDP on healthcare (vs. 10.5% OECD average)  
✅ PoliSim includes 8 healthcare policy models for comparison  
✅ Different policies have different cost, coverage, and efficiency trade-offs  
✅ Multi-year simulations reveal long-term fiscal impacts  

---

## 📚 Next Steps

| Notebook | Topic | Time |
|----------|-------|------|
| **04** | Social Security Deep Dive | 45-60 min |
| **05** | Monte Carlo & Uncertainty | 60-90 min |
| **06** | Tax Policy Modeling | 45-60 min |

---

*Continue to Notebook 04: Social Security Deep Dive* →